In [1]:
import os
import pandas as pd
import yaml
import cx_Oracle
import math
import Levenshtein
from collections import defaultdict

In [2]:
from helpers import bmr, cbac

In [3]:
with open('../../database.yml', 'r') as stream:
    credentials = yaml.load(stream)

login = credentials['production']

connection = cx_Oracle.connect(**login)

In [ ]:
sheet_params = [
    {
        'sheet_name' : 'NSW',
        'name_field' : 'NAME',
        'coords' : ['LONG_GDA94','LAT_GDA94'],
        'string_functions' : [cbac, bmr]
    },{
        'sheet_name' : 'NT',
        'name_field' : 'NAME',
        'coords' : ['LONGITUDE','LATITUDE'],
        'string_functions' : bmr
    },{
        'sheet_name' : 'QLD',
        'name_field' : 'NAME',
        'coords' : ['LONG_GDA94','LAT_GDA_94'],
        'string_functions' : bmr
    },{
        'sheet_name' : 'SA',
        'name_field' : 'NAME',
        'coords' : ['LONG','LAT'],
        'string_functions' : None
    },{
        'sheet_name' : 'VIC (from GA\'s database)',
        'name_field' : 'GSV_NAME',
        'coords' : ['GDA94_LONGITUDE','GDA94_LATITUDE'],
        'string_functions' : None
    },{
        'sheet_name' : 'WA',
        'name_field' : 'NAME',
        'coords' : ['LONG_GDA94','LAT_GDA94'],
        'string_functions' : None
    }
]

In [ ]:
excel = pd.ExcelFile('data/IGSN_STRAT_drillholes.xlsx')

In [ ]:
def find_nearby_holes(long, lat):
    if math.isnan(long) or math.isnan(lat):
        return pd.DataFrame()
    sql = '''select b.borehole_id, b.borehole_name,
                b.location.sdo_point.x as longitude, b.location.sdo_point.y as latitude,
                lubt.text as borehole_type,
                cast(sdo_geom.sdo_distance(b.location,SDO_GEOMETRY(2001,4283,MDSYS.SDO_POINT_TYPE({0}, {1},NULL),NULL,NULL),
                  10, 'unit=M') as number(5)) as distance
                from BOREHOLE.BOREHOLES b
                left outer join borehole.borehole_types bt on bt.BOREHOLE_ID = b.BOREHOLE_ID
                left outer join borehole.lu_borehole_types lubt on lubt.bh_type_code = bt.bh_type_code
                where sdo_within_distance(b.location,
                    SDO_GEOMETRY(2001,4283,MDSYS.SDO_POINT_TYPE({0}, {1}, NULL),NULL,NULL),
                    \'distance=500\') = \'TRUE\''''
    return pd.read_sql(sql.format(long, lat), connection)

In [ ]:
def edit_distance(left, right, fn=None):
    '''Takes two strings and returns the edit distance as a Levenshtein ratio.
    If a function or list of functions are supplied it iterates through each and
    returns the maximum ratio'''
    ratio = Levenshtein.ratio(left, right)
    values = [ratio]
    if fn is None:
        return ratio
    
    if isinstance(fn, list):
        for f in fn:
            values.append(f(left, right))
    else:
        values.append(fn(left, right))
    return max(values)

In [ ]:
def process_data(df, coords, name_field, fn = None):
    result_set = defaultdict(list)
    
    for i, row in enumerate(df.itertuples()):
        if i != 0 and i % 100 == 0:
            print("{0} rows processed ".format(i))
        
        long, lat = getattr(row, coords[0]), getattr(row, coords[1])
        frame = find_nearby_holes(long, lat)
        
        if frame.empty:
            result_set["no_matches"].append(frame)
        else:          
            source_row = pd.DataFrame(data=[row]*len(frame))
            matches = pd.merge(source_row, frame, how='outer', left_index=True, right_index=True)
           
            if name_field:
                matches["EDIT_DISTANCE"] = matches[[name_field,"BOREHOLE_NAME"]].apply(lambda x: edit_distance(str(x[0]).lower(),str(x[1]).lower(), fn), axis = 1)
            matches['CONFIRM'] = ''
            
            
            exact_match = matches[matches["EDIT_DISTANCE"] == 1]
            if len(exact_match) == 1:
                result_set["exact"].append(exact_match)
                
            elif len(exact_match) > 1:
                result_set["multiple"].append(exact_match)
            else:
                if len(matches) == 1:
                    result_set["fuzzy"].append(matches)
                else:
                    result_set["multiple"].append(matches)
                
              
    return {key : pd.concat(result_set[key]) for key in result_set}

In [ ]:
for sheet in excel.sheet_names:
    if sheet == 'GA':
        continue
    else:
        output_path = "OUTPUT_"+sheet
        if not os.path.exists(output_path):
            os.makedirs(output_path)
    parameters = [parameter for parameter in sheet_params if parameter['sheet_name'] == sheet]
    if len(parameters) != 1:
        print(print('No parameters found for worksheet \'{0}\'. Are you sure you set it up?'.format(sheet)))
        continue

    name_field = parameters[0]['name_field']
    coords = parameters[0]['coords']
    fn = parameters[0]['string_functions']
    columns = [name_field] + coords
    df = excel.parse(sheet)[columns]
    results = process_data(df, coords, name_field, fn)

    for key in results:
        pathname = os.path.join(output_path, key.upper() + '.xlsx')
        results[key].to_excel(pathname, index = False)